## BooleanOutputParser
**Yes/No, True/False 형태의 이진 응답**을 처리하는 파서입니다.

**주요 특징:**

- **이진 분류에 최적**: 단순한 예/아니오 판단
- **다양한 표현 인식**: "네", "Yes", "True" 등 다양한 형태 인식
- **Boolean 타입 반환**: Python boolean 값으로 직접 반환
- **간단하고 확실**: 가장 단순하면서도 명확한 결과

**활용 사례:**

- 스팸 메일 판별
- 사실 확인 (Fact Check)
- 승인/거부 결정
- 조건 만족 여부 확인

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import BooleanOutputParser
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Boolean 파서 초기화
parser = BooleanOutputParser()

# 수동으로 포맷 지시사항 정의 (LangChain 버전 이슈 회피)
format_instructions = """
출력은 반드시 다음 중 하나여야 합니다:
- `True`: 모든 조건 충족 시
- `False`: 하나라도 조건 불충족 시

예시:
True  # 모든 조건 만족
False # 조건 불만족
"""

# 승인/거부 결정 프롬프트 템플릿
template = """
다음 대출 신청자를 평가하세요. 조건을 모두 충족하면 `True`, 아니면 `False`를 출력하세요.

### 조건:
1. 나이 >= {min_age}세
2. 신용 점수 >= {min_credit_score}
3. 월 수입 >= ${min_income}

### 신청자 정보:
{applicant_details}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

input_variables=['applicant_details', 'min_age', 'min_credit_score', 'min_income'] input_types={} partial_variables={'format_instructions': '\n출력은 반드시 다음 중 하나여야 합니다:\n- `True`: 모든 조건 충족 시\n- `False`: 하나라도 조건 불충족 시\n\n예시:\nTrue  # 모든 조건 만족\nFalse # 조건 불만족\n'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['applicant_details', 'format_instructions', 'min_age', 'min_credit_score', 'min_income'], input_types={}, partial_variables={}, template='\n다음 대출 신청자를 평가하세요. 조건을 모두 충족하면 `True`, 아니면 `False`를 출력하세요.\n\n### 조건:\n1. 나이 >= {min_age}세\n2. 신용 점수 >= {min_credit_score}\n3. 월 수입 >= ${min_income}\n\n### 신청자 정보:\n{applicant_details}\n\n{format_instructions}\n'), additional_kwargs={})]


In [4]:

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.1
)

# 테스트 케이스
test_cases = [
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 김철수
        - 나이: 25세
        - 신용 점수: 750
        - 월 수입: $3,500
        """
    },
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 이영희
        - 나이: 17세
        - 신용 점수: 680
        - 월 수입: $2,800
        """
    }
]

# 거부 사유 생성 프롬프트 (출력 형식 명시적으로 지정)
reason_template = """
다음 대출 신청 거부 사유를 1문장으로 설명하세요. 반드시 다음 형식으로 답변해야 합니다:

[거부 사유]: [사유 내용]

### 신청자 정보:
{applicant_details}

### 조건:
- 최소 나이: {min_age}세
- 최소 신용 점수: {min_credit_score}
- 최소 월 수입: ${min_income}
"""
reason_prompt = ChatPromptTemplate.from_template(reason_template)
reason_chain = reason_prompt | model | StrOutputParser()

chain = prompt | model | StrOutputParser()

# 체인 확장 (Boolean 파서와 분리)
def get_decision_with_reason(input_dict):
    # 1. 먼저 Boolean 결정
    decision = chain.invoke(input_dict)
    
    # 2. 거부 시에만 사유 생성
    if not decision:
        try:
            reason = reason_chain.invoke({
                "applicant_details": input_dict["applicant_details"],
                "min_age": input_dict["min_age"],
                "min_credit_score": input_dict["min_credit_score"],
                "min_income": input_dict["min_income"]
            })
            return decision, reason
        except Exception as e:
            return decision, f"거부 사유 생성 실패: {str(e)}"
    return decision, "모든 조건을 충족했습니다."

# 테스트 (안전한 실행)
try:
    decision, reason = get_decision_with_reason(test_cases[0])
    print(f"\n결과: {'승인' if decision else '거부'}")
    print(f"사유: {reason}")
except Exception as e:
    print(f"오류 발생: {str(e)}")


결과: 승인
사유: 모든 조건을 충족했습니다.
